In [18]:
__author__ = 'Ksenia Voronaya'

import lxml.html as html
import io
import nltk
import string
import math

In [2]:
# Новостной сайт "Вести"
# http://www.vesti.ru/news/
base_url = 'http://www.vesti.ru'

# Берем три топика с сайта: 
# 1. политика
# 2. спорт
# 3. происшествия
# временной интервал с 01.06.2016 по 05.06.2016
topics = {'politics': 'cid=5', 'sport': 'cid=680', 'accidents': 'cid=8'}

In [3]:
corpus_politics = []
corpus_sport = []
corpus_accidents = []

for topic,cid in topics.iteritems():
    for date in xrange(1, 6):
            main_page = html.parse('http://www.vesti.ru/section.html?' + cid + '&date=0' + str(date) + '.06.2016')
            # Получаем url для новостей определенным топиком
            urls = main_page.getroot().xpath('//div[@class="news-wrapper_section"]//h3//a[@href]/@href')
            for url in urls:
                page = html.parse(base_url + url)
                # Получаем текст статьи
                text = page.getroot().xpath('//div[@class="article__text"]//p/text()')
                # Декодируем текст
                text = [''.join(t).encode('utf-8', 'ignore').decode('utf-8') for t in text]
                page_text = ''.join(text)
                # Удаляем лишние переходы на новые строки и пропуски
                page_text = page_text.replace('\n', ' ').replace('\r', '')
                page_text = " ".join(page_text.split())
                if topic == 'politics':
                    corpus_politics.append(page_text)
                if topic == 'sport':
                    corpus_sport.append(page_text)
                if topic == 'accidents':
                    corpus_accidents.append(page_text)
                    
print("Size of corpus for topic 'politics' is {} texts".format(len(corpus_politics)))
print("Size of corpus for topic 'sport' is {} texts".format(len(corpus_sport)))
print("Size of corpus for topic 'accidents' is {} texts".format(len(corpus_accidents)))

Size of corpus for topic 'politics' is 383 texts
Size of corpus for topic 'sport' is 115 texts
Size of corpus for topic 'accidents' is 191 texts


In [4]:
print(corpus_politics[3])

Кандидат от демократической партии США готова объединиться со своим однопартийцем и конкурентом – все ради того, чтобы вместе противостоять республиканцу Дональду Трампу.Этот вопрос она уже начала обсуждать с представителями Сандерса. Экс-госсекретарь США верит, что ей удастся объединиться со своим конкурентом по праймериз. "Мы оба сделаем все, что сможем, чтобы не пустить Дональда Трампа к Белому дому", — пообещала она. Клинтон и Сандерс, возможно, проведут переговоры по объединению партии 7 июня, сообщает . состоятся 8 ноября 2016 года. От республиканцев в них будет участвовать скандальный миллиардер Дональд Трамп, набравший, несмотря на свои выходки, необходимые 1237 голосов партийных делегатов. Ему будет противостоять кандидат от демократов — либо Клинтон, либо Сандерс.


In [12]:
print(corpus_sport[7])

Главный тренер сборной Хорватии Анте Чачич назвал имена 23-х футболистов, которым предстоит выступить на чемпионате Европы во Франции. В состав команды вошел защитник "Локомотива" Ведран Чорлука.Для тридцатилетнего Чорлуки нынешний чемпионат Европы станет третьим в карьере. Ранее он принимал участие в турнирах 2008 и 2012 годов. Поедет на первенство континента и бывший защитник московского "Динамо" Гордон Шильденфельд, ныне выступающий за другое "Динамо" – загребское.Полностью заявка сборной Хорватии, куда вошли 23 игрока, на Евро-2016 выглядит следующим образом: вратари – Даниел Субашич ("Монако"), Ловре Калинич ("Хайдук"), Иван Варгич ("Риека"); защитники — Дарио Срна ("Шахтер", Украина), Ведран Чорлука ("Локомотив" Москва), Шиме Врсалько ("Сассуоло", Италия), Домагой Вида ("Динамо" Киев, Украина), Иван Стринич ("Наполи", Италия), Гордон Шильденфельд ("Динамо" Загреб), Тин Едвай ("Байер", Германия); полузащитники — Иван Ракитич ("Барселона", Испания), Лука Модрич, Матео Ковачич (оба 

In [24]:
print(corpus_accidents[17])

Жертвами группы хакеров стали шесть российских банков. Полусотня киберпреступников в марте-апреле 2016 года атаковали Металлинвестбанк, Русский международный банк, "Метрополь" и "Регнум". .Операция по нейтрализации хакеров началась после того, как они атаковали одну из дочерних организаций Сбербанка. Но служба кибербезопасности Сбербанка успешно отразила атаку, сообщает . Информацию о попытке внедрения вируса в систему Сбербанк передал в МВД и ФСБ, которые начали специальную операцию.В ее ходе при помощи Лаборатории Касперского было установлено, что преступная группа атаковала банки с помощью банковского трояна Lurk. Киберпреступники получили удаленный доступ к системам Металлинвестбанка. Из этого банка они похитили 680 миллионов рублей, которые перевели на подконтрольные счета.При этом по размерам активов Металлинвестбанк занимает 89 место среди российских банков. Остальные банки, системы которых были поражены вирусом, находятся за пределами первой сотни. У банка "Регнум" лицензия был

In [13]:
# Считываем из файла биграммы и триграммы (НКРЯ)
def load_ngrams(input_file):
    input_f = io.open(input_file, "r", encoding='utf8')
    n_dict = {}
    for line in input_f:
        line = line.replace('\n', '').split('\t')
        n_dict[tuple(line[0:])] = int(line[0])
    input_f.close()
    return n_dict

In [14]:
bigrams = load_ngrams("C:/Users/kvoronaya/Documents/perplexity/2grams-3.txt")
print len(bigrams.keys())

6750525


In [15]:
threegrams = load_ngrams("C:/Users/kvoronaya/Documents/perplexity/3grams-3.txt")
print len(threegrams.keys())

4655170


In [16]:
# Разбиваем корпус по словам
def tokenize(corpus):
    news_list = corpus
    words = []
    for text in news_list:
        text_str = ''.join(text)
        line = nltk.word_tokenize(text_str)
        line = [word.encode("utf-8").decode("utf-8").lower() for word in line if word not in string.punctuation
                        and word not in [u'\u2012', u'\u2013', u'\u2014', u'\u2015', u'``', u"''"]]
        words += line
    return words

In [19]:
tokenize_politics =  tokenize(corpus_politics)

In [26]:
tokenize_sport =  tokenize(corpus_sport)

In [27]:
tokenize_accidents =  tokenize(corpus_accidents)

In [28]:
def find_perplexity(category, n3_gram, n2_gram):
    sum_probability = 0
    unknown_words = 0
    min_probability = 1
    sum_log = 0
    
    current_probability= {}
    
    for i in range(2, len(category)):
        try:
            current_probability[i] = (n3_gram[(category[i-2], category[i-1], category[i])] + 0.0) 
            / n2_gram[(category[i-2],category[i-1])]
            sum_probability += current_probability
            if current_probability[i] < min_probability:
                min_probability = current_probability[i]
        except KeyError:
            unknown_words +=1
    print (unknown_words, len(category), sum_probability)
    known_words = len(category) - unknown_words
    min_probability /= 10.0
    probability_to_substract = (unknown_words * min_probability + 0.0) / known_words
    for i in range(2, len(category)):
        try:
            sum_log += math.log(current_probability[i] - probability_to_substract, 2)
        except KeyError:
            sum_log += math.log(min_probability, 2)

    return sum_log

In [35]:
politics_perplexity = find_perplexity(tokenize_politics, bigrams, threegrams)

perplexity = 2**(-politics_perplexity / len(tokenize_politics))
print("Perplexity for topic 'politics' is {}".format(perplexity))

(88076, 88078, 0)
Perplexity for topic 'politics' is 9.99947716235


In [36]:
sport_perplexity = find_perplexity(tokenize_sport, bigrams, threegrams)

perplexity = 2**(-sport_perplexity / len(tokenize_sport))
print("Perplexity for topic 'sport' is {}".format(perplexity))

(15571, 15573, 0)
Perplexity for topic 'sport' is 9.99704328689


In [34]:
accidents_perplexity = find_perplexity(tokenize_accidents, bigrams, threegrams)

perplexity = 2**(-accidents_perplexity / len(tokenize_accidents))
print("Perplexity for topic 'accidents' is {}".format(perplexity))

(21147, 21149, 0)
Perplexity for topic 'accidents' is 9.99782274868


### Значение perplexity для трех корпусов получилось почти одинаковым. Но все же для корпуса с топиком 'sport' оно чуть меньше, чем для остальных. Поэтому можно сказать, что текущая вероятностная модель корпуса с топиком 'sport' имеет лучшею предсказательную способность, чем остальные две вероятностные модели для корпусов с топиками 'politics' & 'accidents'. 